In [1]:
from gurobipy import *
import pandas as pd
import numpy as np
import csv
import xlsxwriter

In [2]:
N = 500   # 醫護人員數量上限
T = 16    # 日期集合
K = 12    # 行政區集合
J = 456   # 里集合
L = 129   # 候選點集合
Q = 5     # 候選點等級集合 
W = 3     # 疫區等級集合
R = 15    # 候選點等級＋疫區等級組合
# 候選點等級 r 所需的醫療人力
S = [10, 10, 10, 15, 15, 15, 20, 20, 20, 25, 25, 25, 30, 30, 30] 

# Data import
Z = []    # j 里是否在 l 快篩站服務範圍
          # z_jrl

for j in range(J):
    Z.append([])

for q in range(Q):
    for w in range(W):
        path = './Z值/q' + str(q+1) + 'w' + str(w+1) + '.csv'
        with open(path, newline='') as csvfile:
            rows = csv.reader(csvfile)
            for n, row in enumerate(rows):
                if n != 0:
                    row = list(row)
                    for i in range(len(row)):
                        row[i] = int(row[i])
                    Z[n-1].append(row[1:])

path = './風險加權人口.csv'
P = pd.read_csv(path, index_col=0, skiprows=0)  # j里在時間t的風險加權人口
# print(P)

In [3]:
x_writer = pd.ExcelWriter('x_result.xlsx', engine='xlsxwriter')
c_writer = pd.ExcelWriter('c_result.xlsx', engine='xlsxwriter')

for t in range(T):
    # 模型
    F1 = Model('F1')

    # 變數
    # x_lr  # 候選點 l 是否設立候選點等級＋疫區等級組合 r 的站點
    x = []
    for l in range(L):
        x.append([])
        for r in range(R):
            x[l].append(F1.addVar(lb = 0, vtype=GRB.BINARY,
                                          name=('x' + str(l+1) + ',' + str(r+1))))
    # c_j  # j 里有被快篩站覆蓋則為 1，反之為 0
    c = []
    for j in range(J):
        c.append(F1.addVar(lb = 0, vtype=GRB.BINARY, name=('c' + str(j+1))))

    # 目標式
    F1.setObjective(quicksum(c[j] * P.iat[j, t+3] for j in range(J)), GRB.MAXIMIZE)

    # 限制式
    # 第一條限制式刪除：熱區

    # 第二條限制式刪除：熱區

    # 第三條限制式：每個候選點只有一種等級或不被設置
    F1.addConstrs((quicksum(x[l][r] for r in range(R)) <= 1 for l in range(L)))

    # 第四條限制式：醫護人員人力上限
    F1.addConstr(quicksum(quicksum(S[r] * x[l][r] for l in range(L))
                          for r in range(R)) <= N)

    # 第五限制式刪除：熱區(y的定義)

    # 第六條限制式：里是否有在任一快篩站服務範圍內
    F1.addConstrs(quicksum(quicksum(Z[j][r][l] * x[l][r] for l in range(L)) 
                           for r in range(R)) >= c[j] for j in range(J))

    F1.optimize()

    print(f'{t + 1}: {F1.objVal}')
    x_sol_df = pd.DataFrame(index=range(1, 130), columns=range(1, 16))
    c_sol_df = pd.DataFrame(index=range(1, 457), columns=[20210514 + t])
    for l in range(L):
        for r in range(R):
            x_sol_df.iat[l, r] = x[l][r].x
    for j in range(J):
        c_sol_df.iloc[j] = c[j].x
    x_sol_df.to_excel(x_writer, sheet_name=f'{20210514 + t}')
    if t == 0:
        c_sol_df.to_excel(c_writer, header=f'{20210514 + t}')
    else:
        c_sol_df.to_excel(c_writer, startcol=(t+1), index=False, header=f'{20210514 + t}')
x_writer.save()
c_writer.save()

Academic license - for non-commercial use only - expires 2021-08-04
Using license file /Users/yuchiaching/gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 586 rows, 2391 columns and 48433 nonzeros
Model fingerprint: 0xfb2c3f7d
Variable types: 0 continuous, 2391 integer (2391 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [6e+02, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 382 rows and 1903 columns
Presolve time: 0.04s
Presolved: 204 rows, 488 columns, 6126 nonzeros
Variable types: 0 continuous, 488 integer (488 binary)

Root relaxation: objective 7.945143e+05, 50 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     